# Asdex Upgrade Transfer Matrix

In [2]:
using VSTom
using NetCDF

  likely near /home/lstagner/.julia/v0.4/NetCDF/src/NetCDF.jl:438
  likely near /home/lstagner/.julia/v0.4/NetCDF/src/NetCDF.jl:438
 in depwarn at deprecated.jl:73
 in call at deprecated.jl:50
 in include at ./boot.jl:261
 in include_from_node1 at ./loading.jl:304
 in require at ./loading.jl:243
 in include at ./boot.jl:261
 in include_from_node1 at ./loading.jl:304
 in include at ./boot.jl:261
 in include_from_node1 at ./loading.jl:304
 in require at ./loading.jl:243
 in include_string at loading.jl:266
 in execute_request_0x535c5df2 at /home/lstagner/.julia/v0.4/IJulia/src/execute_request.jl:177
 in eventloop at /home/lstagner/.julia/v0.4/IJulia/src/IJulia.jl:141
 in anonymous at task.jl:447
while loading /home/lstagner/.julia/v0.4/NetCDF/src/NetCDF.jl, in expression starting on line 429
  likely near /home/lstagner/.julia/v0.4/NetCDF/src/NetCDF.jl:429
  likely near /home/lstagner/.julia/v0.4/NetCDF/src/NetCDF.jl:429
  likely near /home/lstagner/.julia/v0.4/NetCDF/src/NetCDF.jl:429
 

In [3]:
a = read_ncdf("data/full_synthetic_transfer_matrix.cdf")

transfer_matrix = a["transfer_matrix"]

energy = a["E"] #[keV]
ne_orig =length(energy)
we = energy .< 100.0
energy = energy[we]
dE = abs(energy[2] - energy[1])
nen = length(energy)

pitch = a["p"]
dP = abs(pitch[2] - pitch[1])
np = length(pitch);


nr,nc = size(transfer_matrix)
A = zeros(nr,np*nen)
for i=1:nr
    A[i,:] = reshape(reshape(transfer_matrix[i,:],np,ne_orig)[:,we],1,nen*np)
end

w1 = energy .< 10.0
inds=zeros(Bool,nr)
tol = 0.01 # 1%
for i=1:nr
    wf = reshape(A[i,:],np,nen)
    wf_total = sum(wf)
    if (sum(wf[:,w1])/wf_total < tol) 
        inds[i] = true 
    end
end
A = A[inds,:];

energy_red = collect(linspace(20.0,maximum(energy),sum(energy .> 20.0)))

A = resize_transfer_matrix(A,energy,pitch,energy_red,pitch);

  likely near In[3]:1
  likely near In[3]:1
  likely near In[3]:1
  likely near In[3]:1
  likely near In[3]:1
  likely near In[3]:1
  likely near In[3]:1
  likely near In[3]:1
  likely near In[3]:1
  likely near In[3]:1
  likely near In[3]:1
  likely near In[3]:1
  likely near In[3]:1
  likely near In[3]:1
  likely near In[3]:1
  likely near In[3]:1
  likely near In[3]:1
  likely near In[3]:1
  likely near In[3]:1
  likely near In[3]:1
  likely near In[3]:1
  likely near In[3]:1
  likely near In[3]:1
  likely near In[3]:1
  likely near In[3]:1
  likely near In[3]:1
  likely near In[3]:1
  likely near In[3]:1
  likely near In[3]:1
  likely near In[3]:1
  likely near In[3]:1
  likely near In[3]:1
  likely near In[3]:1
  likely near In[3]:1
  likely near In[3]:1
  likely near In[3]:1
  likely near In[3]:1
  likely near In[3]:1
  likely near In[3]:1
  likely near In[3]:1
  likely near In[3]:1
  likely near In[3]:1
  likely near In[3]:1
  likely near In[3]:1
  likely near In[3]:1
  likely n

In [9]:
np

75

In [4]:
length(energy_red)*length(pitch)

3000

In [5]:
size(A)

(1180,3000)

In [15]:
typeof(t_varid)

NetCDF.NcVar{Float64,2}

In [24]:
using NetCDF

nrows, ncols = size(A)

# Write transfer matrix to file
fname = "data/reduced_synthetic_transfer_matrix.cdf"

isfile(fname) && rm(fname)

nrows_id = NcDim("nrows",nrows)
ncols_id = NcDim("ncols",ncols)
nen_id = NcDim("energy",length(energy_red))
np_id = NcDim("pitch",np)

t_varid = NcVar("transfer_matrix",[nrows_id,ncols_id])
e_varid = NcVar("energy",nen_id)
p_varid = NcVar("pitch",np_id)

ncid = NetCDF.create(fname,NcVar[t_varid,e_varid,p_varid],mode=NC_CLASSIC_MODEL)

NetCDF.putvar(ncid,"transfer_matrix",A)
NetCDF.putvar(ncid,"energy",energy_red)
NetCDF.putvar(ncid,"pitch",pitch)

NetCDF.close(ncid);